In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

This notebook executes the complete estimation procedure required by The Mapinator Classification (Peters and Yu, 2023, work in progress), starting from placements data and ending with model estimates. Various components of this notebook can be altered to create different estimations under different constraints.

This notebook requires a multithreaded kernel; see https://julialang.github.io/IJulia.jl/stable/manual/installation/#Installing-additional-Julia-kernels.

The code for converting API data for use with estimation is adapted from [a notebook](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/2023_summer_projects/api_to_mapinator_julia.ipynb) written by Silas Kwok.

We start by retrieving placements data from the API.

In [ ]:
"""
SBM API Data Filter (Julia Version)
Adapted from James Yuming Yu (5 June 2023)

Silas Kwok, 31 July 2023

Adapted and modified for use with full estimation by James Yu, 17 September 2023
"""

using HTTP, JSON
DEBUG_LEVEL = 1

In [ ]:
function matches(keywords, phrase)
    # checks if any of the keywords are in the phrase
    for keyword in keywords
        if occursin(keyword, phrase)
            return true
        end
    end
    return false
end

### STEP 1a
Retrieve the placement outcomes.

In [ ]:
# NOTE: request times out after 120 seconds. If the data takes longer than 120s to download, adjust the timeout.
placements = nothing
try
    mapinator_data = HTTP.get("https://support.econjobmarket.org/api/mapinator", timeout = 120)
    placements = JSON.parse(String(mapinator_data.body))
catch e
    error("Failed to retrieve data from the API: $e")
end

### STEP 1b
Group placements by applicant ID and eliminate "oid 893" positions (Ocean and Crow).

In [ ]:
# TODO: are the json fields strictly typed? is there a way to easily compensate if the variable types change?

applicant_outcomes = Dict{Any, Vector}()
applicant_ids = Set{Any}()
num_outcomes_selected = 0

for outcome in placements
    push!(applicant_ids, outcome["aid"])
    if outcome["to_oid"] != 893
        push!(get!(applicant_outcomes, outcome["aid"], Vector()), outcome)
        num_outcomes_selected += 1
    end
end

if DEBUG_LEVEL > 0
    println("  ", length(placements), " total placement outcomes")
    println("  -", length(placements) - num_outcomes_selected, " outcomes at Ocean and Crow")
    println("  ", num_outcomes_selected, " outcomes not at Ocean and Crow")
    println()
    println("  ", length(applicant_ids), " total applicants with placements")
    println("  -", length(applicant_ids) - length(applicant_outcomes), " total applicants with exclusively outcomes at Ocean and Crow")
    println("  ", length(applicant_outcomes), " applicants with outcomes not at Ocean and Crow")
end

### STEP 2a
Determine the first placement outcome of each individual that occurred after the individual graduated.\
We need to know what the first outcome is BEFORE we filter on types of outcomes, as otherwise we will get incorrectly-identified "first-time positions".

### STEP 2b
Remove postdoc outcomes so applicants with postdoc positions aren't automatically removed from the data.\
Postdocs are concurrent so the placements are redundant on top of e.g. concurrently-awarded assistant professor positions.

In [ ]:
postdoc_counter = 0
finalized_applicant_outcomes = Dict{Any, Any}()

for applicant_id in keys(applicant_outcomes)
    for outcome in applicant_outcomes[applicant_id]
        # if you wish to display postdocs in the sinks, remove the if statement condition 
        #   and set Post-Doc to have higher priority than Assistant Professor below
        # alternatively, to only include postdocs in the sinks that did not receive professorships,
        #   do not alter the below code, and instead conduct a second pass 
        #   to fill in postdoc outcomes for individuals with no professorships
        if outcome["position_name"] != "Post-Doc"
            if !haskey(finalized_applicant_outcomes, applicant_id)
                # just add the outcome if the applicant doesn't have any yet
                finalized_applicant_outcomes[applicant_id] = outcome
            else
                # otherwise, the applicant does have at least one other outcome
                if outcome["startdate"] < finalized_applicant_outcomes[applicant_id]["startdate"]
                    # take the earliest outcome of the two and ignore the other
                    finalized_applicant_outcomes[applicant_id] = outcome
                elseif outcome["startdate"] == finalized_applicant_outcomes[applicant_id]["startdate"]
                    # sometimes we may have multiple outcomes that started on the same date - follow priority listing
                    if outcome["position_name"] in ["Assistant Professor"]
                        finalized_applicant_outcomes[applicant_id] = outcome
                    elseif outcome["position_name"] in ["Consultant"] && !(finalized_applicant_outcomes[applicant_id]["position_name"] in ["Assistant Professor"])
                        finalized_applicant_outcomes[applicant_id] = outcome
                    elseif outcome["position_name"] in ["Other Academic", "Other Non-Academic"] && !(finalized_applicant_outcomes[applicant_id]["position_name"] in ["Assistant Professor", "Consultant"])
                        finalized_applicant_outcomes[applicant_id] = outcome
                    end
                end
            end
        else
            postdoc_counter += 1
        end
    end
end

if DEBUG_LEVEL > 0
    println("  -", length(applicant_outcomes) - length(finalized_applicant_outcomes), " total applicants removed due to only being postdocs (", 
        postdoc_counter, " total postdoc placements detected)")
    println("  ", length(finalized_applicant_outcomes), " total applicants ported to finalized collection")
end

### STEP 3
Eliminate everything except:
- Assistant Professor
- Consultant
- Other Academic
- Other Non-Academic

In [ ]:
# add "Lecturer" if adjusting sinks later on
valid_labels = Set(["Assistant Professor", "Consultant", "Other Academic", "Other Non-Academic"])
irrelevant_counter = 0
removed_labels = Set()

for applicant_id in copy(keys(finalized_applicant_outcomes))
    outcome = finalized_applicant_outcomes[applicant_id]
    if !(outcome["position_name"] in valid_labels)
        push!(removed_labels, outcome["position_name"])
        delete!(finalized_applicant_outcomes, applicant_id)
        irrelevant_counter += 1
    end
end

if DEBUG_LEVEL > 0
    println("  -", irrelevant_counter, " irrelevant applicants removed from the following classes of positions:")
    println(removed_labels)
    println("  ", length(finalized_applicant_outcomes), " applicants remaining after irrelevant-position applicants removed:")
    maintained_labels = Dict{Any, Int}()
    for applicant_id in keys(finalized_applicant_outcomes)
        outcome = finalized_applicant_outcomes[applicant_id]
        position_name = outcome["position_name"]
        if haskey(maintained_labels, position_name)
            maintained_labels[position_name] += 1
        else
            maintained_labels[position_name] = 1
        end
    end
end

println(maintained_labels, " ", sum(values(maintained_labels)), " total")

### STEP 4
Filter-by-year.

In [ ]:
sorted_by_year = Dict{Any, Dict}()
removed_year_placed = 0

remove_years = [] # ["2022", "2023", "2024", "2025", "2026"] # remove all 2022+ entries

for applicant_id in copy(keys(finalized_applicant_outcomes))
    outcome = finalized_applicant_outcomes[applicant_id]
    if matches(remove_years, outcome["startdate"])
        removed_year_placed += 1
        delete!(finalized_applicant_outcomes, applicant_id)
    else
        push!(get!(sorted_by_year, parse(Int, split(outcome["startdate"], "-")[1]), Dict()), applicant_id => outcome)
    end
end

if DEBUG_LEVEL > 0
    println("  -", removed_year_placed, " applicants removed due to placement in 2022/2023/2024")
    println("  ", length(finalized_applicant_outcomes), " applicants remaining after year corrections")
    println()
end

for key in sort(collect(keys(sorted_by_year)))
    println("Year ", key, " has ", length(sorted_by_year[key]), " placement outcomes")
end

### STEP 5
Save to disk.

In [ ]:
total_check = sum(length(value) for value in values(sorted_by_year))
println("Total " * "$total_check" * " applicants in JSON file (compare to" * " $(length(finalized_applicant_outcomes)) " 
    * "applicants in finalized_applicant_outcomes: " * "$(length(finalized_applicant_outcomes) == total_check ? "SUCCESS" : "FAIL")" * ")")

json_str = JSON.json(sorted_by_year, 4)  
open("to_from_by_year_mapinator_api.json", "w") do f
    write(f, json_str)
end

This concludes the API-loading code. We now have the placements, sorted by year:

In [ ]:
sorted_by_year

We now need to run the type allocation code on these placements. This is adapted from [create_new_allocation.ipynb](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/create_new_allocation.ipynb).

In [ ]:
include("type_allocation_flexible.jl")
included_years = keys(sorted_by_year)
# auto-detect year interval; change this to select the years of data to include in the estimation
YEAR_INTERVAL = minimum(included_years):maximum(included_years)

In [ ]:
to_from_by_year_api = sorted_by_year
# alternatively, load an existing file of placement data
# to_from_by_year_api = SBM_flexible.fetch_data("to_from_by_year.json")

Using the placements, we can sort them into academic and sink placements, as well as collect some labels:

In [ ]:
academic, academic_to, academic_builder, rough_sink_builder, institution_mapping, reverse_mapping = SBM_flexible.get_builders(to_from_by_year_api, YEAR_INTERVAL);

We are mostly flexible in how we choose to design the set of sinks to include. One exception is teaching universities, which must always be included:

In [ ]:
# sink of teaching universities that do not graduate PhDs
# this must be constructed using academic placements, not pre-defined sink placements

teaching_universities = Set() 
for dept_name in academic_to
    if !(dept_name in academic)
        # the department hired an assistant professor but never graduated anyone
        push!(teaching_universities, dept_name)
    end
end

The rest are built by standardized `if` statements:

In [ ]:
public_sector = ("Public Sector", Set())
private_sector = ("Private Sector", Set())
other_groups = ("Other Groups", Set())

postdocs = ("Postdocs", Set())
lecturers = ("Lecturers", Set())
other_academic = ("Other Academic", Set())

for outcome in rough_sink_builder
    if outcome["recruiter_type"] == 5 # government institution
        push!(public_sector[2], (string(outcome["to_name"], " ($(public_sector[1]))"), outcome))
    elseif outcome["recruiter_type"] in [6, 7] # private sector: for and not for profit
        push!(private_sector[2], (string(outcome["to_name"], " ($(private_sector[1]))"), outcome))
    elseif outcome["recruiter_type"] == 8 # international organizations, think tanks, assorted
        push!(other_groups[2], (string(outcome["to_name"], " ($(other_groups[1]))"), outcome))

    # some other examples
    # every example here must also have a corresponding sink Set() above, 
    #     and an entry in sinks_to_include below
   
    elseif outcome["postype"] == 6
        # postdocs that are not in the above (i.e. academic; not public, private, or other)
        # requires reconfiguration of the to_from_by_year loading code
        push!(postdocs[2], (string(outcome["to_name"], " ($(postdocs[1]))"), outcome))
    elseif outcome["postype"] in [5, 7]
        # lecturers that are not in the above
        # requires reconfiguration of the to_from_by_year loading code
        push!(lecturers[2], (string(outcome["to_name"], " ($(lecturers[1]))"), outcome))
    else
        # everything else including terminal academic positions
        # this sink can only be constructed as an "else" statement
        push!(other_academic[2], (string(outcome["to_name"], " ($(other_academic[1]))"), outcome))
    
    end
end

# sort to ensure consistent ordering
academic_list = sort(collect(academic))
teaching_list = sort(collect(teaching_universities))
# to be consistent with the original estimation, we only include these additional sinks:
sinks_to_include = (public_sector, private_sector, other_groups)#, postdocs, lecturers, other_academic)

sink_builder, sinks, sink_labels = SBM_flexible.build_sinks(sinks_to_include, teaching_list)

NUMBER_OF_SINKS = length(sink_labels)
institutions = vcat(academic_list, sinks...)
println("$(length(academic_list)) academic departments, $(length(institutions)) total departments")

Next, the adjacency matrix:

In [ ]:
length(academic_builder) + length(sink_builder)

In [ ]:
out = SBM_flexible.get_adjacency(academic_list, institutions, academic_builder, sink_builder);

In [ ]:
sum(out)

We are now ready to run the SBM type allocation itself. We want to determine the optimal number of types along the way, which we adapt from [estimate_number_of_types.jl](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/estimate_number_of_types.jl).

In [ ]:
using Optim, Random
Random.seed!(0)

Functions to solve for the optimal number of types:

In [ ]:
function β(K, likelihoods, λ, numtotal_test, n)
    return likelihoods[K] - λ * ((K * numtotal_test) / 2) * n * log(n)
end

function w(β_vec)
    return β_vec ./ sum(β_vec)
end

function objective_to_minimize(λ_vec, range_K, likelihoods, numsink, n)
    λ = λ_vec[1] # just 1-D
    w_vec = w([β(K, likelihoods, λ, K + numsink, n) for K in range_K])
    return sum(w_vec .* log.(w_vec))
end

Compute the likelihoods:

In [ ]:
likelihoods = Dict{}()
range_K = 1:10

Threads.@threads for NUMBER_OF_TYPES_TEST in range_K
    println("Executing K = $NUMBER_OF_TYPES_TEST types on thread $(Threads.threadid())")
    Random.seed!(0) # for reproducibility
    numtotal_test = NUMBER_OF_TYPES_TEST + NUMBER_OF_SINKS
    
    @time est_obj, est_alloc = SBM_flexible.doit(out, length(academic_list), [length(s) for s in sinks], NUMBER_OF_TYPES_TEST, numtotal_test, 500 * (NUMBER_OF_TYPES_TEST-2) + 1000)
    _, _, _, full_likelihood = SBM_flexible.get_allocation(est_alloc, out, NUMBER_OF_TYPES_TEST, numtotal_test, institutions)
    likelihoods[NUMBER_OF_TYPES_TEST] = full_likelihood
    println("Completed K = $NUMBER_OF_TYPES_TEST with likelihood $full_likelihood")
end

Apply functions to obtain the optimal number of types:

In [ ]:
res = optimize(λ -> objective_to_minimize(λ, range_K, likelihoods, NUMBER_OF_SINKS, length(institutions)), [0.0])
println("Optimal hyperparameter:")
# TODO: figure out whether there is a potential issue of a non-singleton argmax set (the argmax needs to be the highest lambda in the set)
display(Optim.minimizer(res)[1])
candidate_types = [β(K, likelihoods, Optim.minimizer(res)[1], K + NUMBER_OF_SINKS, length(institutions)) for K in range_K]
println()
println("penalized likelihoods: ", candidate_types)
println("maximum value: ", maximum(candidate_types))
println(" optimal number of types which maximizes the penalized likelihood: ", argmax(candidate_types))

From here, we re-run the type allocation with the optimal number of types to obtain some initial estimates.

In [ ]:
NUMBER_OF_TYPES = argmax(candidate_types)
# NUMBER_OF_TYPES = 4 # can also hard-code this
numtotal = NUMBER_OF_TYPES + NUMBER_OF_SINKS

In [ ]:
Random.seed!(0) # for reproducibility
@time est_obj, est_alloc = SBM_flexible.doit(out, length(academic_list), [length(s) for s in sinks], NUMBER_OF_TYPES, numtotal, 500 * (NUMBER_OF_TYPES-2) + 1000)

In [ ]:
placement_rates, counts, sorted_allocation, full_likelihood = SBM_flexible.get_allocation(est_alloc, out, NUMBER_OF_TYPES, numtotal, institutions)

We can explore the results:

In [ ]:
placement_rates

In [ ]:
placement_rates ./ counts # means

In [ ]:
full_likelihood

In [ ]:
SBM_flexible.nice_table(placement_rates, NUMBER_OF_TYPES, NUMBER_OF_SINKS, sink_labels)

To save the allocation to file, if we want to explore it later, we can do the following:

In [ ]:
type_dictionary = []
for (i, alloc) in enumerate(sorted_allocation)
    if alloc in 1:NUMBER_OF_TYPES
        inst_id = reverse_mapping[institutions[i]]
        push!(type_dictionary, Dict("name" => institutions[i], "institution_id" => inst_id, "type" => alloc))
    end
end

In [ ]:
open(".estimates/id_to_type_api.json", "w") do f
    write(f, JSON.json(type_dictionary))
end;

We print out the type allocation at the end of the notebook, as it is large.

Now that we have the type allocation and placement rates, we can estimate the underlying model parameters. We draw from two sources, the [chi-square](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/estimate_model_two_rounds.jl) and [maximum likelihood](https://github.com/jbrightuniverse/mapinator/blob/master/estimation/estimate_model_ml.jl) variants.

In [ ]:
using BlackBoxOptim, Distributions, ForwardDiff, Integrals, Roots, StatsPlots, DelimitedFiles

Helper functions for the mathematics:

In [ ]:
# define p_vec as e.g.
# [v2/v1 v3/v2 v4/v3 α1 α2 α3 α4 mu1 mu2 mu3 mu4 mus sg1 sg2 sg3 sg4 sgs]

# truncated normal: https://juliastats.org/Distributions.jl/stable/truncate/#Distributions.TruncatedNormal
# TODO: update to the most recent notation so the package version can be updated

function F(x, ρ, μ, σ, K)
    return sum([ρ[i] * cdf(TruncatedNormal(μ[i], σ[i], 0, 1), x) for i in 1:K])
end

function f(x, ρ, μ, σ, K)
    return sum([ρ[i] * pdf(TruncatedNormal(μ[i], σ[i], 0, 1), x) for i in 1:K])
end

function G(Fim1, Fx, αsum)
    return (Fim1 - Fx) / αsum
end

function κ(i, t, v_rel)
    return sum([-log(v_rel[j]) for j in t:i-1])
end

function fi(x, μ, σ)
    return pdf(TruncatedNormal(μ, σ, 0, 1), x)
end

function q(i, t, Fx_vec, x_vec, ρ, μ, σ, α, v_rel, k, K)
    # Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    # x_vec[s] = x_{s-1}, so the limits of integration are and must be offset by 1 below
    # TODO: can some integrals be cached as a speed-up? can some integrals be computed in parallel?
    return sum([(α[t]/sum(α[1:s])) * solve(IntegralProblem{false}((x, p) -> exp(-(G(Fx_vec[s], F(x, ρ, μ, σ, K), sum(α[1:s])) + κ(s, t, v_rel))) * fi(x, μ[i], σ[i]), x_vec[s+1], x_vec[s]), HCubatureJL())[1] for s in t:k])
end

function Fx(t, α, v_rel)
    return 1 - sum([-log(v_rel[j])*sum(α[1:j]) for j in 1:t])
end

function Q2(ratio, β)
    return β * (1 - exp(-ratio)) / ratio
end

function pi(t, α)
    return α[t] / sum(α[1:t])
end

We can use one of two possible estimators:

In [ ]:
function chisquare(p_vec, placements, k, K, M)
    v_rel = p_vec[1:k-1]
    α = 1.0 * p_vec[k:2k-1] / sum(p_vec[k:2k-1]) # change 1.0 to scale number of graduates vs departments
    μ = p_vec[2k:2k+K-1]
    σ = p_vec[2k+K:2k+2K-1]
    ρ = p_vec[2k+2K:2k+3K-1] / sum(p_vec[2k+2K:2k+3K-1])
    β = p_vec[end] # set = 1 to hardcode
    
    Fx_vec = ones(k) # sets F(x0) = 1 by default; Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    x_vec = ones(k+1) # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0 # TODO: if this case occurs, can we speed up q()?
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        # there is no simple closed-form for F^{-1}(x) so this numerically computes x1, x2, x3
        x_vec[t+1] = find_zero(x -> F(x, ρ, μ, σ, K) - Fx_vec[t+1], 0.5) 
    end
    
    objective = 0.0
    normalizer = 0.0
    
    q_it = zeros(K, k)
    for i in 1:K, t in 1:k
        prob = q(i, t, Fx_vec, x_vec, ρ, μ, σ, α, v_rel, k, K)
        q_it[i, t] = prob
        normalizer += ρ[i] * prob
    end
    
    D = sum([ρ[i] * (1 - sum([q_it[i, t] for t in 1:k])) for i in 1:K])
    S = sum([α[t] * (1 - sum([ρ[i]*q_it[i, t] for i in 1:K])) for t in 1:k])
    round_2 = zeros(K, k)
    for i in 1:K, t in 1:k
        prob = (1 - sum([q_it[i, s] for s in 1:k])) * Q2(D/S, β) * α[t] * (1 - sum([ρ[j] * q_it[j, t] for j in 1:K])) / S
        round_2[i, t] = prob
        normalizer += ρ[i] * prob
    end
    
    # TODO: div by zero and negative floating point edge cases in mean
    for i in 1:K, t in 1:k 
        expectation = M * (ρ[i] * (q_it[i, t] + round_2[i, t]) / normalizer)
        objective += (placements[i, t] - expectation) ^ 2 / expectation
    end

    return objective
end

function print_metrics_chisquare(p_vec, placements, k, K, M)
    v_rel = p_vec[1:k-1]
    α = 1.0 * p_vec[k:2k-1] / sum(p_vec[k:2k-1]) # change 1.0 to scale number of graduates vs departments
    μ = p_vec[2k:2k+K-1]
    σ = p_vec[2k+K:2k+2K-1]
    ρ = p_vec[2k+2K:2k+3K-1] / sum(p_vec[2k+2K:2k+3K-1])
    β = p_vec[end] # set = 1 to hardcode

    Fx_vec = ones(k)
    x_vec = ones(k+1)
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        x_vec[t+1] = find_zero(x -> F(x, ρ, μ, σ, K) - Fx_vec[t+1], 0.5) 
    end

    objective = 0.0
    normalizer = 0.0
    
    q_it = zeros(K, k)
    for i in 1:K, t in 1:k
        prob = q(i, t, Fx_vec, x_vec, ρ, μ, σ, α, v_rel, k, K)
        q_it[i, t] = prob
        normalizer += ρ[i] * prob
    end
    
    D = sum([ρ[i] * (1 - sum([q_it[i, t] for t in 1:k])) for i in 1:K])
    S = sum([α[t] * (1 - sum([ρ[i]*q_it[i, t] for i in 1:K])) for t in 1:k])
    round_2 = zeros(K, k)
    round_2_hiring = zeros(K, k)

    for i in 1:K, t in 1:k
        prob = (1 - sum([q_it[i, s] for s in 1:k])) * Q2(D/S, β) * α[t] * (1 - sum([ρ[j] * q_it[j, t] for j in 1:K])) / S
        round_2[i, t] = prob
        round_2_hiring[i, t] = Q2(D/S, β) * α[t] * (1 - sum([ρ[j] * q_it[j, t] for j in 1:K])) / S
        normalizer += ρ[i] * prob
    end

    round_1_failure = zeros(K)
    for i in 1:K
        round_1_failure[i] = (1 - sum([q_it[i, s] for s in 1:k]))
    end
    
    # TODO: div by zero and negative floating point in mean
    exp_placements_round_1 = zeros(K, k)
    exp_placements_round_2 = zeros(K, k)
    exp_placements = zeros(K, k)
    for i in 1:K, t in 1:k 
        expectation = M * (ρ[i] * (q_it[i, t] + round_2[i, t]) / normalizer)
        exp_placements_round_1[i, t] = M * (ρ[i] * q_it[i, t] / normalizer)
        exp_placements_round_2[i, t] = M * (ρ[i] * round_2[i, t] / normalizer)
        exp_placements[i, t] = expectation
        objective += (placements[i, t] - expectation) ^ 2 / expectation
    end

    println("objective value = ", objective)
    println("success sample size (departments) = ", M)
    println("estimated total samples (departments) = ", M / normalizer)
    println("estimated unmatched departments = ", (M / normalizer) - M)
    println("probability of any success: ", normalizer)
    println("probability of no success: ", 1 - normalizer)
    println("measure of departments in round 2 = ", D)
    println("measure of graduates in round 2 = ", S)
    println()
    println("predicted fraction of departments of each tier:")
    display(ρ)
    println()
    println("fractions observed among successful departments in data:")
    display(sum(placements, dims = 2) ./ M)
    println()

    for i in 1:k
        println("pi_", i, " = ", pi(i, α))
    end
    println()

    offer_targets = zeros(k, k)
    for t in 1:k, j in 1:t
        offer_targets[t, j] = pi(j, α) * prod([1 - pi(i, α) for i in j+1:t])
    end
    println("Tier selection probabilities for making offers:")
    display(offer_targets)
    println()

    println("Round 1 hiring probabilities:")
    display(q_it)
    println()

    println("Probabilities of failing round 1")
    display(round_1_failure)
    println()

    println("Probabilities of failing round 1 and hiring in round 2:")
    display(round_2)
    println()

    println("Round 2 hiring probabilities:")
    display(round_2_hiring)
    println()

    for i in 1:k+1
        println("x_", i - 1, " = ", x_vec[i])
    end
    println()
    for i in 1:k
        println("F(x_", i - 1, ") = ", Fx_vec[i])
    end
    println()
    for i in 1:k
        println("α_", i, " = ", α[i])
        println("  Est. graduates: ", α[i] * (M / normalizer - 1))
        println("  Successful: ", sum(placements[:, i]))
        println("  Unsuccessful: ", (α[i] * (M / normalizer - 1)) - sum(placements[:, i]))
    end
    println("Total estimated graduates: ", sum(α) * (M / normalizer - 1))
    println("Total successful graduates: ", M)
    println("Total estimated unsuccessful graduates: ", (sum(α) * (M / normalizer - 1)) - M)
    println("β = ", β)
    println()
    println("estimated placement rates, round 1 only:")
    display(exp_placements_round_1)
    println()
    println("estimated placement rates, round 2 only:")
    display(exp_placements_round_2)
    println()
    println("estimated placement rates, cumulative:")
    display(exp_placements)
    println()
    println("actual placement rates:")
    display(placements)
    println()
    println("difference between estimated and actual placement rates:")
    display(exp_placements - placements)
    println()
    println("chi-square p-value")
    println(1 - cdf(Chisq((size(placements)[1] - 1) * (size(placements)[2] - 1)), objective))
end


In [ ]:
function estimate_likelihood(p_vec, placements, k, K, M)
    v_rel = p_vec[1:k-1]
    α = 1.0 * p_vec[k:2k-1] / sum(p_vec[k:2k-1]) # change 1.0 to scale number of graduates vs departments
    μ = p_vec[2k:2k+K-1]
    σ = p_vec[2k+K:2k+2K-1]
    ρ = p_vec[2k+2K:2k+3K-1] / sum(p_vec[2k+2K:2k+3K-1])
    β = p_vec[end] # set = 1 to hardcode
    
    Fx_vec = ones(k) # sets F(x0) = 1 by default; Fx_vec = [F(x0)=1, F(x1), F(x2), F(x3), ..., F(xk-1)]
    x_vec = ones(k+1) # x_vec = [x0 = 1, x1, x2, x3, ..., xk = 0]
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0 # TODO: if this case occurs, can we speed up q()?
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        # there is no simple closed-form for F^{-1}(x) so this numerically computes x1, x2, x3
        x_vec[t+1] = find_zero(x -> F(x, ρ, μ, σ, K) - Fx_vec[t+1], 0.5) 
    end

    objective = 0.0
    normalizer = 0.0

    q_it = zeros(K, k)
    for i in 1:K
        for t in 1:k
            prob = q(i, t, Fx_vec, x_vec, ρ, μ, σ, α, v_rel, k, K)
            q_it[i, t] = prob
            normalizer += ρ[i] * prob
        end
    end

    # TODO: div by zero and negative floating point in mean
    D = sum([ρ[i] * (1 - sum([q_it[i, t] for t in 1:k])) for i in 1:K])
    S = sum([α[t] * (1 - sum([ρ[i]*q_it[i, t] for i in 1:K])) for t in 1:k])
    for i in 1:K
        for t in 1:k
            prob = (1 - sum([q_it[i, s] for s in 1:k])) * Q2(D/S, β) * α[t] * (1 - sum([ρ[j] * q_it[j, t] for j in 1:K])) / S
            # println(sum([q_it[i, s] for s in 1:k]))
            # println(t, " ", i, " ", ρ[i] * (q_it[i, t] + prob), " ", ρ[i], " ", q_it[i, t], " ", (1 - sum([q_it[i, s] for s in 1:k])), " ", Q2(D/S, β), " ", α[t], " ", (1 - sum([ρ[j] * q_it[j, t] for j in 1:K])), " ", S, " ", prob)
            objective += placements[i, t] * (log(ρ[i] * (q_it[i, t] + prob)))
            normalizer += ρ[i] * prob
        end
    end
    
    objective -= M * log(normalizer)
    return -objective
end

function print_metrics_maximum_likelihood(p_vec, placements, k, K, M)
    v_rel = p_vec[1:k-1]
    α = 1.0 * p_vec[k:2k-1] / sum(p_vec[k:2k-1]) # change 1.0 to scale number of graduates vs departments
    μ = p_vec[2k:2k+K-1]
    σ = p_vec[2k+K:2k+2K-1]
    ρ = p_vec[2k+2K:2k+3K-1] / sum(p_vec[2k+2K:2k+3K-1])
    β = p_vec[end] # set = 1 to hardcode

    Fx_vec = ones(k)
    x_vec = ones(k+1)
    x_vec[k+1] = 0.0
    for t in 1:k-1
        Fx_vec_candidate = Fx(t, α, v_rel)
        if Fx_vec_candidate <= 0.0
            Fx_vec[t+1:k] .= 0.0
            x_vec[t+1:k] .= 0.0
            break
        end
        Fx_vec[t+1] = Fx_vec_candidate
        x_vec[t+1] = find_zero(x -> F(x, ρ, μ, σ, K) - Fx_vec[t+1], 0.5) 
    end

    objective = 0.0
    likelihood = 0.0
    normalizer = 0.0
    
    q_it = zeros(K, k)
    for i in 1:K, t in 1:k
        prob = q(i, t, Fx_vec, x_vec, ρ, μ, σ, α, v_rel, k, K)
        q_it[i, t] = prob
        normalizer += ρ[i] * prob
    end
    
    # TODO: div by zero and negative floating point in mean
    D = sum([ρ[i] * (1 - sum([q_it[i, t] for t in 1:k])) for i in 1:K])
    S = sum([α[t] * (1 - sum([ρ[i]*q_it[i, t] for i in 1:K])) for t in 1:k])
    round_2 = zeros(K, k)
    round_2_hiring = zeros(K, k)

    for i in 1:K, t in 1:k
        prob = (1 - sum([q_it[i, s] for s in 1:k])) * Q2(D/S, β) * α[t] * (1 - sum([ρ[j] * q_it[j, t] for j in 1:K])) / S
        round_2[i, t] = prob
        round_2_hiring[i, t] = Q2(D/S, β) * α[t] * (1 - sum([ρ[j] * q_it[j, t] for j in 1:K])) / S
        normalizer += ρ[i] * prob
    end

    round_1_failure = zeros(K)
    for i in 1:K
        round_1_failure[i] = (1 - sum([q_it[i, s] for s in 1:k]))
    end
    
    exp_placements_round_1 = zeros(K, k)
    exp_placements_round_2 = zeros(K, k)
    exp_placements = zeros(K, k)
    for i in 1:K, t in 1:k 
        expectation = M * (ρ[i] * (q_it[i, t] + round_2[i, t]) / normalizer)
        exp_placements_round_1[i, t] = M * (ρ[i] * q_it[i, t] / normalizer)
        exp_placements_round_2[i, t] = M * (ρ[i] * round_2[i, t] / normalizer)
        exp_placements[i, t] = expectation
        objective += (placements[i, t] - expectation) ^ 2 / expectation
        likelihood += placements[i, t] * (log(ρ[i] * (q_it[i, t] + round_2[i, t])))
        likelihood -= log(factorial(big(placements[i, t])))
    end
    likelihood -= M * log(normalizer)

    println("chi-square objective value = ", objective)
    println("log-likelihood objective value = ", likelihood)
    println("likelihood objective value = ", exp(likelihood))
    println("success sample size (departments) = ", M)
    println("estimated total samples (departments) = ", M / normalizer)
    println("estimated unmatched departments = ", (M / normalizer) - M)
    println("probability of any success: ", normalizer)
    println("probability of no success: ", 1 - normalizer)
    println("measure of departments in round 2 = ", D)
    println("measure of graduates in round 2 = ", S)
    println()
    println("predicted fraction of departments of each tier:")
    display(ρ)
    println()
    println("fractions observed among successful departments in data:")
    display(sum(placements, dims = 2) ./ M)
    println()

    for i in 1:k
        println("pi_", i, " = ", pi(i, α))
    end
    println()

    offer_targets = zeros(k, k)
    for t in 1:k, j in 1:t
        offer_targets[t, j] = pi(j, α) * prod([1 - pi(i, α) for i in j+1:t])
    end
    println("Tier selection probabilities for making offers:")
    display(offer_targets)
    println()

    println("Round 1 hiring probabilities:")
    display(q_it)
    println()

    println("Probabilities of failing round 1")
    display(round_1_failure)
    println()

    println("Probabilities of failing round 1 and hiring in round 2:")
    display(round_2)
    println()

    println("Round 2 hiring probabilities:")
    display(round_2_hiring)
    println()

    for i in 1:k+1
        println("x_", i - 1, " = ", x_vec[i])
    end
    println()
    for i in 1:k
        println("F(x_", i - 1, ") = ", Fx_vec[i])
    end
    println()
    for i in 1:k
        println("α_", i, " = ", α[i])
        println("  Est. graduates: ", α[i] * (M / normalizer - 1))
        println("  Successful: ", sum(placements[:, i]))
        println("  Unsuccessful: ", (α[i] * (M / normalizer - 1)) - sum(placements[:, i]))
    end
    println("Total estimated graduates: ", sum(α) * (M / normalizer - 1))
    println("Total successful graduates: ", M)
    println("Total estimated unsuccessful graduates: ", (sum(α) * (M / normalizer - 1)) - M)
    println("β = ", β)
    println()
    println("estimated placement rates, round 1 only:")
    display(exp_placements_round_1)
    println()
    println("estimated placement rates, round 2 only:")
    display(exp_placements_round_2)
    println()
    println("estimated placement rates, cumulative:")
    display(exp_placements)
    println()
    println("actual placement rates:")
    display(placements)
    println()
    println("difference between estimated and actual placement rates:")
    display(exp_placements - placements)
    println()
    println("chi-square p-value")
    println(1 - cdf(Chisq((size(placements)[1] - 1) * (size(placements)[2] - 1)), objective))
end

We can then use our earlier results to produce the estimates:

In [ ]:
Random.seed!(0) # for reproducibility
res = (;placements = placement_rates)
# res = (;placements = [496 66 33 3; 455 196 84 12; 725 618 363 43; 84 148 83 63; 126 66 41 29; 359 258 131 34; 447 206 94 18; 394 523 508 143]) # can also load an adjacency matrix directly

In [ ]:
k = NUMBER_OF_TYPES
K = numtotal
M = sum(res.placements)

# upper bound on the value ratios, which should all be less than 1
# if any ratio turns out to be 1.0 or close to it at optimality, this could indicate that a lower tier has a higher value than a higher one
upper = [1.0 for _ in 1:k-1] 

# upper bound on variables proportionate to alpha
append!(upper, [1.0 for _ in 1:k])

# upper bound on the mu parameter of truncated normal, which is strictly within [0, 1] as the mean is greater than mu in truncated normal
append!(upper, [1.0 for _ in 1:K])

# upper bound on the sigma parameter of truncated normal
append!(upper, [5.0 for _ in 1:K])

# upper bound on variables proportionate to rho
append!(upper, [1.0 for _ in 1:K])

# upper bound on beta friction parameter
push!(upper, 1.0)

# all lower bounds are zero as these should be positive parameters
# can swap estimate_likelihood for chi_square
sol_res = bboptimize(p -> estimate_likelihood(p, res.placements, k, K, M), SearchRange = [(0.0, upper[i]) for i in eachindex(upper)], MaxFuncEvals = 100000, TraceInterval = 5)
sol = best_candidate(sol_res)

In [ ]:
# ensure the correct metrics function is used based on the selected optimizer
print_metrics_maximum_likelihood(sol, res.placements, k, K, M)

for i in 1:k-1
    println("v_", i + 1, "/v_", i, " = ", sol[i])
end

v_base = 1
for i in 1:k
    println("v", i, ": ", v_base)
    if i != k
        v_base = sol[i] * v_base
    end
end

for select_type in 1:k
    println("mean for type ", select_type, ": ", mean(TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1)))
    println("stddev for type ", select_type, ": ", std(TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1)))
    println()
end
for select_type in k+1:K
    println("mean for sink ", select_type - k, ": ", mean(TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1)))
    println("stddev for sink ", select_type - k, ": ", std(TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1)))
    println()
end

We can now plot the CDFs and PDFs.

In [ ]:
# https://github.com/JuliaPlots/StatsPlots.jl/blob/master/README.md
# https://docs.juliaplots.org/latest/tutorial/

select_type = 1
cdfs = plot(TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1), func = cdf, title = "CDFs of Types", label = "Type 1")
for select_type in 2:NUMBER_OF_TYPES # academic types
    plot!(cdfs, TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1), func = cdf, label = string("Type ", select_type))
end

for select_type in k+1:K # sinks
    plot!(cdfs, TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1), func = cdf, label = string("Sink ", select_type - k))
end
xlabel!(cdfs, "offer value")
ylabel!(cdfs, "F(offer value)")
savefig(cdfs, "cdfs.png")
cdfs

In [ ]:
select_type = 1
pdfs = plot(TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1), func = pdf, title = "PDFs of Types", label = "Type 1")
for select_type in 2:k # academic types
    plot!(pdfs, TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1), func = pdf, label = string("Type ", select_type))
end

for select_type in k+1:K # sinks
    plot!(pdfs, TruncatedNormal(sol[2k-1+select_type], sol[2k-1+select_type+K], 0, 1), func = pdf, label = string("Sink ", select_type - k))
end
xlabel!(pdfs, "offer value")
ylabel!(pdfs, "f(offer value)")
savefig(pdfs, "pdfs.png")
pdfs

Below is the coarse-grained type allocation from before:

In [ ]:
for sorted_type in 1:NUMBER_OF_TYPES+NUMBER_OF_SINKS
    counter = 0
    inst_hold = []
    println("TYPE $sorted_type:")
    for (i, sbm_type) in enumerate(sorted_allocation)
        if sbm_type == sorted_type
            push!(inst_hold, institutions[i])
            counter += 1
        end
    end
    for inst in sort(inst_hold)
        println("  ", inst)
    end
    println("Total Institutions: $counter")
    println()
end

One more thing we can do is to estimate a fine-grained department ranking using the estimated values.

In [ ]:
values = ones(k)
v_base = 1
for i in 1:k-1
    v_base = sol[i] * v_base
    values[i+1] = v_base
end
values

We can rank departments by the total expected value of their graduates:

In [ ]:
valued_indices = []
for i in 1:size(out)[2]
    total_graduate_value = values[sorted_allocation[i]] * sum(out[:, i])
    push!(valued_indices, (institutions[i], total_graduate_value))
end
sort!(valued_indices, by = x -> x[2], rev = true)
for (j, entry) in enumerate(valued_indices)
    println("$(j). $(entry[1]) ($(entry[2]))")
end

or by the total expected value of their hires:

In [ ]:
valued_indices_hire = []
for i in 1:size(out)[2]
    total_hire_value = 0
    for j in 1:size(out)[2]
        total_hire_value += values[sorted_allocation[j]] * out[i, j]
    end
    push!(valued_indices_hire, (institutions[i], total_hire_value))
end
sort!(valued_indices_hire, by = x -> x[2], rev = true)
for (j, entry) in enumerate(valued_indices_hire)
    println("$(j). $(entry[1]) ($(entry[2]))")
end